In [ ]:
from user_constants import *
import tpu_simulation_utilities
import tensorflow.compat.v1 as tf
import numpy as np
import os
import io
import sys
from matplotlib.colors import LightSource
import matplotlib.pyplot as plt
from skimage import transform
import ipywidgets as widgets
from osgeo import gdal
from typing import Any, Callable, Iterable, Sequence, List, Dict, Mapping, MutableMapping, MutableSequence, Optional, Text, Tuple, Union

if PUBLIC_COLAB:
  # Authenticate to access Google Cloud Storage.
  from google.colab import auth  # pylint: disable=g-import-not-at-top
  auth.authenticate_user()

def soliton_1d_benchmark(y, x, L, nx, ny, dem):
    A = 15
    lamb = 40150
    a2 = 4.5
    a1 = a2 / 3
    x1 = 50500 - (0.5151125 * lamb + 500)

    x2 = 50500 - (0.2048 * lamb + 500)		#	Leading Depression Nwave
  #	x2 = 1000 + 0.825425 * L 	#	Leading Elevation Nwave

    k1 = 28.416 / (lamb ** 2)
    k2 = 256 / (lamb ** 2)
    z = 2 * (a1 * tf.math.exp( -k1 * tf.math.square(y - x1) ) - a2 * tf.math.exp( -k2 * tf.math.square(y - x2)))
    water_level = 0.1 * 50000
    z = z + water_level*(1-(y/50000))
    z = tf.where(y >50000, tf.ones_like(z)*SAINT_VENANT_EPS,z)
    z = tf.expand_dims(z,axis=0)
    z = tf.repeat(z, nx,axis = 0)
    return z

def run_simulation(
    dem_tiff_filename: Text, resolution: int, num_secs: float,
    num_secs_per_cycle: float, dt: float, cx: int, cy: int,
    run_dir: Text, start_time_secs: float = 0) -> None:
  """Runs the simulation."""
  # Necessary Component of Running Simulation, no editing here.
  slope = 1e-4
  input_file_format = ('{}' + f'-{start_time_secs}.np' if start_time_secs > 0
                       else None)

  unpadded_dem,geoinfo = tpu_simulation_utilities.load_dem_from_tiff_file(dem_tiff_filename,resolution)
  unpadded_river_mask = np.ones_like(unpadded_dem, dtype=bool)
  unpadded_manning_matrix = tpu_simulation_utilities.get_manning_matrix_from_river_mask(
      unpadded_river_mask, MANNING_COEFF_RIVER, MANNING_COEFF_FLOODPLAIN)
  params = tpu_simulation_utilities.get_sv_params(unpadded_dem.shape, resolution, num_secs,
                         num_secs_per_cycle, cx, cy, dt)
  print('params:', params)
  
  # SETS THE BOUNDARY CONDITIONS FOR HEIGHT, FLUX IN Y, FLUX IN X
  h_bcs = [
      tpu_simulation_utilities.NeumannBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.LEFT,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0*3940,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix),
      tpu_simulation_utilities.NeumannBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.RIGHT,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix)
  ]

  q_y_bcs = [
      tpu_simulation_utilities.NeumannBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.LEFT,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix),
      tpu_simulation_utilities.DirichletBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.RIGHT,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix)
  ]

  q_x_bcs = [
     tpu_simulation_utilities.NeumannBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.TOP,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix),
      tpu_simulation_utilities.NeumannBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.BOTTOM,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix)]

  # NO NEED TO EDIT THE FOLLOWING
  init_files_manager = tpu_simulation_utilities.InitFilesManager(
      params, tpu_simulation_utilities.three_d_subgrid_of_2d_grid, run_dir)

  # LAST TWO ARGUMENTS SET INITIAL CONDITIONS IN HEIGHT AND FLUX IN Y DIRECTION
  sim_builder = tpu_simulation_utilities.get_sim_builder(
      params, unpadded_dem, unpadded_manning_matrix,h_bcs,q_x_bcs,q_y_bcs,
      init_files_manager, input_file_format, start_time_secs, 
      soliton_1d_benchmark, tpu_simulation_utilities.constant_height_zeros)

  def session_runner_builder(sim):
    return tpu_simulation_utilities.get_session_runner_builder(params, init_files_manager, run_dir, sim)

  manager = tpu_simulation_utilities.TPUSimulationManager(params, sim_builder, session_runner_builder)
  manager.run_simulation(start_time_secs)

In [ ]:
# SIMULATION PARAMETERS - NEED TO BE SET
resolution, dt, cx = 4, 100000, 5e-3, 1
start_time_secs, num_secs, num_secs_per_cycle = 0, 400.0,10.0
# SIMULATION DEM FILE NAME
dem_bucket_filename = f'gs://{BUCKET}/isec-benchmark-slope.tif'

# NO NEED TO EDIT THIS COMPONENT
dem_tiff_filename = tpu_simulation_utilities.get_dem_tiff_filename(dem_bucket_filename,resolution)
run_dir = f'gs://{BUCKET}'
if PUBLIC_COLAB:
  cy = 8
  os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
  TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

In [ ]:
# RUN SIMULATION ON TPU
if PUBLIC_COLAB:
  g = tf.Graph()
  run_meta = tf.RunMetadata()
  with g.as_default():
    run_simulation(dem_tiff_filename, resolution, num_secs,
                  num_secs_per_cycle, dt, cx, cy, run_dir,
                  start_time_secs)

In [ ]:
# Load the DEM and the heightmaps.
dem, geoinfo = tpu_simulation_utilities.load_dem_from_tiff_file(dem_tiff_filename,resolution)
dem_shape = dem.shape
heightmaps = {}
t_range = np.arange(start_time_secs,num_secs + num_secs_per_cycle,num_secs_per_cycle)
for t in t_range:
  t  = round(t,3)
  filename = os.path.join(run_dir, f'h-{t}.np')
  print(f'Loading heightmap: h-{t}.np')
  with tf.io.gfile.GFile(filename, 'rb') as f:
    heightmaps[t] = np.load(f)
  
resize_factor = 1
resize_shape = [dem_shape[0] // resize_factor, dem_shape[1] // resize_factor]
scaled_dem = transform.resize(dem, resize_shape)
scaled_heightmaps = {}

for ts, hmap in heightmaps.items():
  scaled_heightmaps[ts] = transform.resize(hmap, resize_shape)

In [ ]:
%matplotlib inline
import pandas as pd
import numpy.ma
# For this method to work, the file corresponding to the ISEC benchmark files must be uploaded as well
def plot_benchmark(time_secs,ax=None):
  if (ax == None):
    plt.figure(figsize=(8, 8), dpi=100)
    ax = plt.gca()
  t = time_secs
  row=10
  x = np.linspace(0,50500,50501)
  ax.fill_between(x,np.zeros_like(scaled_dem[row,:]), scaled_dem[row,:],color='gray',alpha=0.25,label='Topography')
  ax.plot(x, scaled_dem[row,:] + scaled_heightmaps[t][row,:],color = '#7570b3',label='Computed')
  if (abs(time_secs - 160)  <= 1 or abs(time_secs - 175)  <= 1 or abs(time_secs - 220)  <= 1 ):
    dataset = pd.read_csv(f'/content/t{time_secs}.csv')
    ax.plot(50000 - dataset['x(m)'],dataset['eta(m)']+5000,label='ISEC Benchmark',color='#fc8d62',linestyle='dashed')
  ax.legend(loc='upper left')
  ax.set_title('Water Height at t = ' + str(t) + ' seconds')
  ax.set_xlim([49000, 50300])
  ax.set_ylim([4975, 5025])

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(15,3),dpi=100)
plot_benchmark(160.0,ax[0])
plot_benchmark(175.0,ax[1])
plot_benchmark(220.0,ax[2])